Important aspects of EV Charging to focus on:
- charging efficiency
- station availability
- station reliability
- cost of charging
- charger comptabiliity
- range anxiety

In [12]:
import datetime
import time
import requests
import json
import re
import numpy as np
import pandas as pd
import seaborn as sns

# Reddit API
import praw

# Natural Language Processing (NLP)
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Topic Modeling
from gensim import corpora
from gensim.models import LdaModel, Phrases
from gensim.models.phrases import Phraser

# Visualization
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# Transformers
from transformers import pipeline

# Miscellaneous
from pprint import pprint
from IPython import display


In [2]:
# # Initialize Reddit API
# reddit = praw.Reddit(
#     client_id=open('cred/client_id.txt').read().strip(),
#     client_secret=open('cred/client_secret.txt').read().strip(),
#     user_agent=open('cred/user_agent.txt').read().strip(),
# )

# # Define Reddit post URL
# reddit_post_url = "https://www.reddit.com/r/electricvehicles/comments/1e7x13p/it_is_not_the_evs_that_are_lacking_in_the_us_its/"
# submission = reddit.submission(url=reddit_post_url)

# # Create a list to hold post and comment data
# post_data = {
#     'post_id': submission.id,
#     'post_title': submission.title,
#     'post_text': submission.selftext,
#     'post_score': submission.score,
#     'post_author': submission.author.name if submission.author else "Deleted",
# }

# comments_data = []

# # Fetch comments and store them
# submission.comments.replace_more(limit=None)

# # Scrape comments
# for comment in submission.comments.list():
#     comments_data.append({
#         'comment_id': comment.id,
#         'author': comment.author.name if comment.author else "Deleted",
#         'score': comment.score,
#         'comment_text': comment.body
#     })

# # Create DataFrames
# post_df = pd.DataFrame([post_data])  # The post data will be a single row
# comments_df = pd.DataFrame(comments_data)

# # Display the data (Post Content and Comments)
# print("Post Data:")
# print(post_df.head())  # Only showing post info (first row)
# print("\nComments Data:")
# print(comments_df.head())  # Display the first few comments

# # Optionally: Save to CSV or JSON if needed
# post_df.to_csv("post_data.csv", index=False)
# comments_df.to_csv("comments_data.csv", index=False)

# # Sleep for 60 seconds before scraping again
# time.sleep(60)  # Adjust as needed

In [3]:
# aspects = [
#     "Charging stations",
#     "Charging network",
#     "Range anxiety",
#     "Charging speed",
#     "Charger type",
#     "Availability of chargers",
#     "Cost of charging",
#     "Maintenance issues",
#     "Tesla charging network",
#     "EV range"
# ]


# # preprocess comments: Convert to lowercase and remove special characters
# def preprocess_text(text):
#     text = text.lower()  # Convert to lowercase
#     text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with one
#     text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation and special characters
#     return text

# # categorise comments depending on aspects identified above
# def categorize_comment(comment_text, aspects):
#     categories = []
#     for aspect in aspects:
#         if aspect.lower() in comment_text:
#             categories.append(aspect)
#     return categories


# # -------------------------------------------------------------------------------------------


# categorized_comments = []

# for comment in comments_df.itertuples():
#     comment_text = preprocess_text(comment.comment_text)  # Preprocess the comment text
#     categories = categorize_comment(comment_text, aspects)
    
#     # Store the result along with the original comment data
#     categorized_comments.append({
#         'comment_id': comment.comment_id,
#         'author': comment.author,
#         'score': comment.score,
#         'comment_text': comment.comment_text,
#         'categories': categories
#     })

# # Create a DataFrame for categorized comments
# categorized_comments_df = pd.DataFrame(categorized_comments)

# # Display categorized comments

# pd.set_option('display.max_rows', None)
# print(categorized_comments_df)



In [4]:
# category_counts = categorized_comments_df['categories'].explode().value_counts()
# print(category_counts)

# SECOND TRY.

In [5]:
# from gensim.models import KeyedVectors

# # Load Google's pre-trained Word2Vec model (download it if needed)
# # This is a large model, so you may want to store it locally and load it
# # model = KeyedVectors.load_word2vec_format('path/to/GoogleNews-vectors-negative300.bin', binary=True)

# # For this example, we’ll use a smaller, faster model from Gensim
# from gensim.models import Word2Vec

# # Train a small model on your dataset if you don't want to use a pre-trained model
# # Example using your comments data
# from nltk.tokenize import word_tokenize
# import nltk
# nltk.download('punkt')

# # Tokenize all comments (lowercase)
# tokenized_comments = [word_tokenize(comment.lower()) for comment in comments_df['comment_text']]

# # Train Word2Vec model on the tokenized data
# word2vec_model = Word2Vec(tokenized_comments, vector_size=100, window=5, min_count=1, sg=0)
# word2vec_model.save("word2vec.model")


In [6]:
# from gensim.models import Word2Vec
# import pandas as pd
# import numpy as np
# from nltk.tokenize import word_tokenize
# import nltk
# nltk.download('punkt')

# # Assuming you already have the comments_df
# comments_data = comments_df['comment_text']

# # Load pre-trained Word2Vec model (or your own)
# word2vec_model = Word2Vec.load("word2vec.model")

# # Define broad theme words based on your categories
# theme_words = {
#     'satisfaction': ['good', 'excellent', 'love', 'great', 'enjoy'],
#     'frustration': ['bad', 'slow', 'delay', 'frustrating', 'worse'],
#     'charging_stations': ['station', 'stations', 'location', 'public station', 'charging point', 'infrastructure', 'charger', 'kiosk', 'EV station', 'public chargers', 'station network', 'station availability'],
#     'charging_network': ['network', 'connected', 'networked', 'charging grid', 'network coverage', 'roaming', 'charging locations', 'map', 'network reliability', 'network expansion', 'partner network'],
#     'range_anxiety': ['range', 'range anxiety', 'range fear', 'battery life', 'battery capacity', 'distance', 'travel range', 'anxiety', 'worry', 'trip range', 'unable to charge', 'running out of charge', 'mileage'],
#     'charging_speed': ['fast', 'slow', 'speed', 'charging rate', 'fast-charging', 'quick', 'fast charging', 'charging speed', 'time to charge', 'slow charging', 'quick charge', 'fast charger', 'fast charging stations'],
#     'charger_type': ['charger', 'type', 'level 1', 'level 2', 'DC fast charging', 'supercharger', 'home charger', 'wall box', 'charging adapter', 'charging port', 'connector', 'plug type', 'Type 1', 'Type 2', 'CCS', 'CHAdeMO'],
#     'availability_of_chargers': ['available', 'not available', 'missing', 'rare', 'insufficient', 'availability', 'location', 'access', 'scarce', 'scarce charging', 'find chargers', 'nearby', 'accessible', 'not available', 'out of service', 'open station', 'charger access', 'charger shortage'],
#     'cost_of_charging': ['cost', 'price', 'expensive', 'affordable', 'cheap', 'price per kWh', 'electricity cost', 'costly', 'charging fees', 'rates', 'price of charging', 'cost to charge', 'free charging', 'charging cost', 'pricing model', 'pricing scheme', 'cost per session'],
#     'maintenance_issues': ['maintenance', 'repair', 'broken', 'malfunction', 'service', 'failure', 'out of service', 'maintenance required', 'charger broken', 'charger error', 'service required', 'maintenance costs', 'down time', 'maintenance issues'],
#     'tesla_charging_network': ['Tesla', 'supercharger', 'Tesla chargers', 'Tesla network', 'Tesla charging', 'supercharger station', 'Tesla charging stations', 'Tesla infrastructure', 'Tesla owners', 'Tesla charging speed'],
#     'ev_range': ['EV', 'range', 'battery', 'miles', 'distance', 'range per charge', 'battery life', 'vehicle range', 'driving range', 'range capacity', 'charge range', 'range efficiency']
# }


# # Helper function to find the most similar theme based on Word2Vec embeddings
# def get_similarity_score(comment, theme_words, model):
#     # Tokenize the comment
#     tokens = word_tokenize(comment.lower())
    
#     # Calculate average vector for the comment
#     vectors = [model.wv[word] for word in tokens if word in model.wv]
    
#     if not vectors:
#         return None
    
#     # Calculate the average vector for the comment
#     avg_comment_vector = np.mean(vectors, axis=0)
    
#     # Create a dictionary to store the similarity score for each theme
#     similarity_scores = {}
    
#     for theme, words in theme_words.items():
#         theme_vectors = [model.wv[word] for word in words if word in model.wv]
#         avg_theme_vector = np.mean(theme_vectors, axis=0)
        
#         # Compute cosine similarity between the comment vector and theme vector
#         similarity_scores[theme] = np.dot(avg_comment_vector, avg_theme_vector) / (np.linalg.norm(avg_comment_vector) * np.linalg.norm(avg_theme_vector))
    
#     return similarity_scores

# # Now process the comments and categorize based on the most relevant theme
# categorized_comments = []

# for comment in comments_data:
#     similarity_scores = get_similarity_score(comment, theme_words, word2vec_model)
    
#     if similarity_scores:
#         # Find the most relevant theme by selecting the highest similarity score
#         most_relevant_theme = max(similarity_scores, key=similarity_scores.get)
#         categorized_comments.append(most_relevant_theme)
#     else:
#         categorized_comments.append("uncategorized")

# # Add categorized themes to your DataFrame
# comments_df['theme'] = categorized_comments

# # Display the categorized comments

# print(comments_df.head())

# # Optionally: Save to CSV or JSON if needed
# comments_df.to_csv("categorized_comments.csv", index=False)


# Third try.

In [14]:
# Initialize Reddit API
reddit = praw.Reddit(
    client_id=open('cred/client_id.txt').read().strip(),
    client_secret=open('cred/client_secret.txt').read().strip(),
    user_agent=open('cred/user_agent.txt').read().strip(),
)

# Define Reddit post URL
reddit_post_url = "https://www.reddit.com/r/electricvehicles/comments/1e7x13p/it_is_not_the_evs_that_are_lacking_in_the_us_its/"
submission = reddit.submission(url=reddit_post_url)

# Define the theme words for each aspect
theme_words = {
    'charging_stations': ['station', 'stations', 'location', 'public station', 'charging point', 'infrastructure', 'charger', 'kiosk', 'EV station', 'public chargers', 'station network', 'station availability'],
    'charging_network': ['network', 'connected', 'networked', 'charging grid', 'network coverage', 'roaming', 'charging locations', 'map', 'network reliability', 'network expansion', 'partner network'],
    'range_anxiety': ['range', 'range anxiety', 'range fear', 'battery life', 'battery capacity', 'distance', 'travel range', 'anxiety', 'worry', 'trip range', 'unable to charge', 'running out of charge', 'mileage'],
    'charging_speed': ['fast', 'slow', 'speed', 'charging rate', 'fast-charging', 'quick', 'fast charging', 'charging speed', 'time to charge', 'slow charging', 'quick charge', 'fast charger', 'fast charging stations'],
    'availability_of_chargers': ['available', 'availability', 'location', 'access', 'scarce', 'scarce charging', 'find chargers', 'nearby', 'accessible', 'not available', 'out of service', 'open station', 'charger access', 'charger shortage'],
    'cost_of_charging': ['cost', 'price', 'expensive', 'affordable', 'cheap', 'price per kWh', 'electricity cost', 'charging fees', 'rates', 'price of charging', 'cost to charge', 'free charging', 'charging cost', 'pricing model', 'pricing scheme', 'cost per session'],
    'maintenance_issues': ['maintenance', 'repair', 'broken', 'malfunction', 'service', 'failure', 'out of service', 'maintenance required', 'charger broken', 'charger error', 'service required', 'maintenance costs', 'down time', 'maintenance issues'],
    'tesla_charging_network': ['Tesla', 'supercharger', 'Tesla chargers', 'Tesla network', 'Tesla charging', 'supercharger station', 'Tesla charging stations', 'Tesla infrastructure', 'Tesla owners', 'Tesla charging speed'],
    'ev_range': ['EV', 'range', 'battery', 'miles', 'distance', 'range per charge', 'battery life', 'vehicle range', 'driving range', 'range capacity', 'charge range', 'range efficiency'],
    

    'vehicle model': ['tesla model 3', 'hyundai kona', 'nissan leaf', 'chevy bolt', 'bmw i3'],
    'charging_station_location': ['houston', 'san francisco', 'los angeles', 'chicago', 'new york'],
    'charger_type': ['charger', 'type', 'level 1', 'level 2', 'DC fast charging', 'DC fast charger', 'supercharger', 'home charger', 'wall box', 'charging adapter', 'charging port', 'connector', 'plug type', 'Type 1', 'Type 2', 'CCS', 'CHAdeMO', 'L2', 'L1'],
}

# Create a list to hold post and comment data
post_data = {
    'post_id': submission.id,
    'post_title': submission.title,
    'post_text': submission.selftext,
    'post_score': submission.score,
    'post_author': submission.author.name if submission.author else "Deleted",
}

comments_data = []

# Fetch comments and store them
submission.comments.replace_more(limit=None)

# Function to categorize a comment into multiple themes
def categorize_comment(comment_text, theme_synonyms):
    categorized_themes = []
    for theme, synonyms in theme_synonyms.items():
        for synonym in synonyms:
            if re.search(r'\b' + re.escape(synonym) + r'\b', comment_text, re.IGNORECASE):
                categorized_themes.append(theme)
                break
    return categorized_themes


lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)


# Scrape comments and categorize them
for comment in submission.comments.list():
    comment_text = comment.body
    matched_themes = categorize_comment(comment_text, theme_words)
    
    comments_data.append({
        'comment_id': comment.id,
        'author': comment.author.name if comment.author else "Deleted",
        'score': comment.score,
        'comment_text': comment_text,
        'themes': matched_themes  # Store all matched themes
    })

# Create DataFrames
post_df = pd.DataFrame([post_data])  # The post data will be a single row
comments_df = pd.DataFrame(comments_data)

# Display the data (Post Content and Comments)
print("Post Data:")
print(post_df.head())  # Only showing post info (first row)
print("\nComments Data:")
print(comments_df.head())  # Display the first few comments

# Optionally: Save to CSV or JSON if needed
post_df.to_csv("post_data.csv", index=False)
comments_df.to_csv("comments_data.csv", index=False)

# Sleep for 60 seconds before scraping again
time.sleep(60)  # Adjust as needed


Post Data:
   post_id                                         post_title  \
0  1e7x13p  It is not the EVs, that are lacking in the US,...   

                                           post_text  post_score post_author  
0  So, I own a ICE vehicle, but often will rent E...         373  Ok-Pea3414  

Comments Data:
  comment_id              author  score  \
0    le387km             Deleted    116   
1    le39p6l          Dirks_Knee     62   
2    le3j257  iwantthisnowdammit     30   
3    le4yq2n          iSeerStone      9   
4    le3icon           StLandrew      8   

                                        comment_text  \
0  We need a lot more L2 AC to reduce the need of...   
1  You're kinda the inverse example which highlig...   
2  We’re in the “before USB outlets on airplanes”...   
3  Chargers should be:\n1. Maintained like gas pu...   
4  I have to record this for posterity. "I own a ...   

                                              themes  
0                                

In [15]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from transformers import pipeline

In [16]:
# Initialize Reddit API
reddit = praw.Reddit(
    client_id=open('cred/client_id.txt').read().strip(),
    client_secret=open('cred/client_secret.txt').read().strip(),
    user_agent=open('cred/user_agent.txt').read().strip(),
)

# Define Reddit post URL
reddit_post_url = "https://www.reddit.com/r/electricvehicles/comments/1e7x13p/it_is_not_the_evs_that_are_lacking_in_the_us_its/"
submission = reddit.submission(url=reddit_post_url)

# Define the theme words for each aspect (same as your current code)
theme_words = {
    'charging_stations': ['station', 'stations', 'location', 'public station', 'charging point', 'infrastructure', 'charger', 'kiosk', 'EV station', 'public chargers', 'station network', 'station availability'],
    'charging_network': ['network', 'connected', 'networked', 'charging grid', 'network coverage', 'roaming', 'charging locations', 'map', 'network reliability', 'network expansion', 'partner network'],
    'range_anxiety': ['range', 'range anxiety', 'range fear', 'battery life', 'battery capacity', 'distance', 'travel range', 'anxiety', 'worry', 'trip range', 'unable to charge', 'running out of charge', 'mileage'],
    'charging_speed': ['fast', 'slow', 'speed', 'charging rate', 'fast-charging', 'quick', 'fast charging', 'charging speed', 'time to charge', 'slow charging', 'quick charge', 'fast charger', 'fast charging stations'],
    'availability_of_chargers': ['available', 'availability', 'location', 'access', 'scarce', 'scarce charging', 'find chargers', 'nearby', 'accessible', 'not available', 'out of service', 'open station', 'charger access', 'charger shortage'],
    'cost_of_charging': ['cost', 'price', 'expensive', 'affordable', 'cheap', 'price per kWh', 'electricity cost', 'charging fees', 'rates', 'price of charging', 'cost to charge', 'free charging', 'charging cost', 'pricing model', 'pricing scheme', 'cost per session'],
    'maintenance_issues': ['maintenance', 'repair', 'broken', 'malfunction', 'service', 'failure', 'out of service', 'maintenance required', 'charger broken', 'charger error', 'service required', 'maintenance costs', 'down time', 'maintenance issues'],
    'tesla_charging_network': ['Tesla', 'supercharger', 'Tesla chargers', 'Tesla network', 'Tesla charging', 'supercharger station', 'Tesla charging stations', 'Tesla infrastructure', 'Tesla owners', 'Tesla charging speed'],
    'ev_range': ['EV', 'range', 'battery', 'miles', 'distance', 'range per charge', 'battery life', 'vehicle range', 'driving range', 'range capacity', 'charge range', 'range efficiency'],
    'vehicle model': ['tesla model 3', 'hyundai kona', 'nissan leaf', 'chevy bolt', 'bmw i3'],
    'charging_station_location': ['houston', 'san francisco', 'los angeles', 'chicago', 'new york'],
    'charger_type': ['charger', 'type', 'level 1', 'level 2', 'DC fast charging', 'DC fast charger', 'supercharger', 'home charger', 'wall box', 'charging adapter', 'charging port', 'connector', 'plug type', 'Type 1', 'Type 2', 'CCS', 'CHAdeMO', 'L2', 'L1'],
}

# Create a list to hold post and comment data
post_data = {
    'post_id': submission.id,
    'post_title': submission.title,
    'post_text': submission.selftext,
    'post_score': submission.score,
    'post_author': submission.author.name if submission.author else "Deleted",
}

comments_data = []

# Fetch comments and store them
submission.comments.replace_more(limit=None)

# Initialize BERT-based sentiment analysis model
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Function to categorize a comment into multiple themes using BERT-based model
def categorize_comment_bert(comment_text, candidate_labels):
    # Using zero-shot classification to predict themes
    result = classifier(comment_text, candidate_labels)
    return result['labels']

lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

# Scrape comments and categorize them
for comment in submission.comments.list():
    comment_text = comment.body
    matched_themes = categorize_comment_bert(comment_text, list(theme_words.keys()))  # Using BERT for classification
    
    comments_data.append({
        'comment_id': comment.id,
        'author': comment.author.name if comment.author else "Deleted",
        'score': comment.score,
        'comment_text': comment_text,
        'themes': matched_themes  # Store all matched themes
    })

# Create DataFrames
post_df = pd.DataFrame([post_data])  # The post data will be a single row
comments_df = pd.DataFrame(comments_data)

# Display the data (Post Content and Comments)
print("Post Data:")
print(post_df.head())  # Only showing post info (first row)
print("\nComments Data:")
print(comments_df.head())  # Display the first few comments

# Optionally: Save to CSV or JSON if needed
post_df.to_csv("post_data.csv", index=False)
comments_df.to_csv("comments_data.csv", index=False)

# Sleep for 60 seconds before scraping again
time.sleep(60)  # Adjust as needed


KeyboardInterrupt: 

In [19]:
import praw

# Initialize Reddit API
reddit = praw.Reddit(
    client_id=open('cred/client_id.txt').read().strip(),
    client_secret=open('cred/client_secret.txt').read().strip(),
    user_agent=open('cred/user_agent.txt').read().strip(),
)

# Define a list of keywords related to EV charging
keywords = ['charging stations', 'EV charging', 'Tesla charging', 'range anxiety', 'charging cost']

# Define a subreddit to search in
subreddit = reddit.subreddit('electricvehicles')  # or 'evcharging', 'EV', etc.

# Fetch top posts or new posts based on keywords
posts_to_scrape = []

# Search posts for specific keywords and filter by engagement
for submission in subreddit.search(" OR ".join(keywords), sort='top', time_filter='all', limit=10):
    if submission.num_comments > 10 and submission.score > 100:  # Filter based on popularity
        posts_to_scrape.append(submission.url)

# Print the URLs of posts to scrape
print("Posts to scrape:", posts_to_scrape)


Posts to scrape: ['https://www.reddit.com/r/electricvehicles/comments/1eqk69a/tesla_is_not_a_luxury_vehicle/', 'https://www.reddit.com/r/electricvehicles/comments/152536h/as_a_conservative_i_hated_the_idea_of_owning_an/', 'https://www.reddit.com/r/electricvehicles/comments/13r4skr/bought_a_very_cheap_ev_two_weeks_ago_and_its/', 'https://www.reddit.com/r/electricvehicles/comments/1ekjapv/psa_avoid_the_chevrolet_blazer_ev/', 'https://www.reddit.com/r/electricvehicles/comments/16df0wz/ill_never_understand_naysayers/', 'https://www.reddit.com/r/electricvehicles/comments/12hn48z/five_years_of_model_3_ownership_by_the_numbers/', 'https://www.reddit.com/gallery/z4x6ve', 'https://i.redd.it/tndowc3qt7ad1.png', 'https://www.reddit.com/r/electricvehicles/comments/18zam6a/my_ev_is_now_10_years_old_pros_and_cons_of_owning/', 'https://i.redd.it/ousp1fmu2cg81.jpg']


In [ ]:
import praw

# Initialize Reddit API
reddit = praw.Reddit(
    client_id=open('cred/client_id.txt').read().strip(),
    client_secret=open('cred/client_secret.txt').read().strip(),
    user_agent=open('cred/user_agent.txt').read().strip(),
)

# Define a list of keywords related to vehicle models and cities
keywords = [
    "Tesla Model 3", "Hyundai Kona", "Nissan Leaf", "Chevy Bolt", "BMW i3",
    "Houston EV charging", "San Francisco EV stations", "Los Angeles charging network", 
    "Chicago Tesla Superchargers", "New York public chargers"
]

# Define a subreddit to search in
subreddit = reddit.subreddit('electricvehicles')  # You can add others like 'evcharging' or 'teslamotors'

# Fetch top posts or new posts based on keywords
posts_to_scrape = []

# Search posts for specific keywords and filter by engagement
for submission in subreddit.search(" OR ".join(keywords), sort='top', time_filter='all', limit=10):
    if submission.num_comments > 10 and submission.score > 100:  # Filter based on popularity
        posts_to_scrape.append(submission.url)

# Print the URLs of posts to scrape, each on a new line
print("Posts to scrape:")
for post in posts_to_scrape:
    print(post)
